In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [73]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical column and see what happens off of this kind of naive approach.

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [74]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [75]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [5]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [71]:
pd.get_dummies(y2015)

It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [8]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

array([0.9801952 , 0.98036143, 0.98105008, 0.98188122, 0.96706246,
       0.97520779, 0.95001069, 0.98088295, 0.98007457, 0.9799316 ])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [77]:
for i in range(len(y2015.loan_status)):
    if y2015.at[i, 'loan_status'] == 'Fully Paid':
        y2015.at[i, 'loan_status'] = 1
    else:
        y2015.at[i, 'loan_status'] = 0

In [78]:
y2015.loan_status = pd.to_numeric(y2015.loan_status)

In [47]:
y2015.corr().loc[:, 'loan_status'].sort_values(ascending=False)

loan_status                       1.000000
last_pymnt_amnt                   0.753773
total_rec_prncp                   0.611635
total_pymnt                       0.503723
total_pymnt_inv                   0.503716
num_tl_op_past_12m                0.071918
acc_open_past_24mths              0.071736
open_acc_6m                       0.065110
open_il_12m                       0.063359
open_il_24m                       0.062345
inq_last_12m                      0.060340
mort_acc                          0.057780
total_cu_tl                       0.057367
total_acc                         0.053061
open_rv_24m                       0.045515
avg_cur_bal                       0.045476
bc_open_to_buy                    0.044218
open_rv_12m                       0.043258
il_util                           0.041305
tot_cur_bal                       0.039465
num_rev_accts                     0.039406
num_bc_tl                         0.038957
tot_hi_cred_lim                   0.038666
mths_since_

In [57]:
y2015 = y2015[['loan_status', 'total_rec_prncp', 'num_tl_op_past_12m',
              'open_il_24m', 'inq_last_12m', 'mort_acc', 'total_cu_tl',
              'total_acc', 'avg_cur_bal', 'bc_open_to_buy', 'il_util',
               'tot_cur_bal', 'num_rev_accts', 'num_bc_tl',
               'tot_hi_cred_lim', 'mths_since_recent_bc_dlq',
              'mths_since_recent_revol_delinq', 'inq_last_6mths',
              'pub_rec_bankruptcies', 'mths_since_last_record', 'inq_fi',
               'funded_amnt', 'dti_joint', 'mths_since_recent_inq',
               'mths_since_rcnt_il', 'all_util', 'mo_sin_rcnt_tl',
               'num_actv_bc_tl', 'num_actv_rev_tl', 'recoveries',
               'collection_recovery_fee', 'bc_util', 'id', 'out_prncp_inv']]

In [58]:
y2015.corr()

,loan_status,total_rec_prncp,num_tl_op_past_12m,open_il_24m,inq_last_12m,mort_acc,total_cu_tl,total_acc,avg_cur_bal,bc_open_to_buy,...,mths_since_rcnt_il,all_util,mo_sin_rcnt_tl,num_actv_bc_tl,num_actv_rev_tl,recoveries,collection_recovery_fee,bc_util,id,out_prncp_inv
loan_status,1.000000,0.611635,0.071918,0.062345,0.060340,0.057780,0.057367,0.053061,0.045476,0.044218,...,-0.041563,-0.046485,-0.048684,-0.052339,-0.057560,-0.064921,-0.065185,-0.087121,-0.119724,-0.519706
total_rec_prncp,0.611635,1.000000,-0.008880,0.020792,0.021394,0.178249,0.067704,0.146773,0.183546,0.188208,...,-0.014422,-0.055942,0.014739,0.098766,0.056092,-0.100933,-0.100695,-0.030997,-0.188353,-0.135268
num_tl_op_past_12m,0.071918,-0.008880,1.000000,0.446085,0.453532,0.048155,0.106768,0.335309,-0.073257,0.074575,...,-0.265792,-0.036096,-0.490789,0.163368,0.272512,0.042770,0.044234,-0.174605,0.008886,-0.082727
open_il_24m,0.062345,0.020792,0.446085,1.000000,0.262772,0.070657,0.271978,0.327416,0.010171,-0.018155,...,-0.483421,0.228108,-0.269002,-0.037062,-0.016627,0.034189,0.035663,-0.045258,0.001371,-0.010739
inq_last_12m,0.060340,0.021394,0.453532,0.262772,1.000000,0.088937,0.067801,0.200231,0.039247,-0.001841,...,-0.188328,0.037285,-0.238692,0.046445,0.094142,0.030557,0.032616,-0.088955,-0.001412,-0.028346
mort_acc,0.057780,0.178249,0.048155,0.070657,0.088937,1.000000,0.169887,0.356387,0.452144,0.125294,...,-0.044964,-0.039673,-0.048839,0.042265,0.061998,-0.002563,-0.002358,-0.009168,-0.013036,0.123226
total_cu_tl,0.057367,0.067704,0.106768,0.271978,0.067801,0.169887,1.000000,0.264322,0.047476,-0.006059,...,-0.138082,0.043523,-0.085295,-0.074398,0.017216,0.012112,0.010824,-0.009783,-0.000914,0.026913
total_acc,0.053061,0.146773,0.335309,0.327416,0.200231,0.356387,0.264322,1.000000,0.042199,0.222279,...,-0.181232,-0.003639,-0.212142,0.298630,0.408414,0.017125,0.018079,-0.103282,-0.014479,0.100473
avg_cur_bal,0.045476,0.183546,-0.073257,0.010171,0.039247,0.452144,0.047476,0.042199,1.000000,0.042099,...,-0.026535,0.095831,0.043938,-0.110135,-0.153915,-0.003055,-0.004201,0.076760,0.007556,0.128512
bc_open_to_buy,0.044218,0.188208,0.074575,-0.018155,-0.001841,0.125294,-0.006059,0.222279,0.042099,1.000000,...,0.001788,-0.468038,-0.042936,0.254651,0.115924,-0.008055,-0.010985,-0.539107,0.022195,0.087487


In [59]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

array([0.87242935, 0.99966754, 0.99943006, 0.99874139, 0.99696034,
       0.99829019, 0.99878886, 0.98976466, 0.99774395, 0.99259048])

In [79]:
y2015 = y2015[['loan_status', 'total_rec_prncp', 'num_tl_op_past_12m',
              'open_il_24m', 'inq_last_12m', 'mort_acc', 'total_cu_tl',
               'num_actv_bc_tl', 'num_actv_rev_tl', 'recoveries',
               'collection_recovery_fee', 'bc_util', 'id', 'out_prncp_inv']]

In [80]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

array([0.83324626, 0.96575635, 0.95366896, 0.95065305, 0.94478746,
       0.93215388, 0.92196443, 0.90246741, 0.9346458 , 0.9484421 ])